# Optimal Power Flow: IEEE 33 Bus Case with Pandapower
- Pandapower OPF 결과 도출
- Pandapower 참고: https://dwightreid.com/site/power-system-contingency-analysis-with-python-pandapower/
- IEEE 33 bus 데이터 구조
  - https://matpower.org/docs/ref/matpower6.0/case33bw.html

1. 계통 불러오기

In [13]:
import pandapower as pp
import pandapower.networks as pn
import pandas as pd
import numpy as np

net = pn.case33bw()
#net.shunt['in_service'] = False
pp.runpp(net,numba=False)
base_MVA = net._ppc['baseMVA']


2. Data 형식
- Bus Data
- Branch Data
- Gen Data
- Load Data
- Y Bus and Connectivity data
- Etc (나중에 고려)

In [14]:
#Bus
Bus_info = pd.DataFrame(net.bus[['name','vn_kv','max_vm_pu','min_vm_pu','type','zone','geo']])
if 0 == Bus_info['name'][0]:
    Bus_info['name'] = Bus_info['name'].values + 1
tmp = Bus_info['name']
tmp.name = 'Buses' 
tmp.to_csv('./Pre_cal_data/Buses.csv',index=False) # For Pyomo Sets

Bus_info.set_index('name',inplace=True)
Bus_info.index.name = 'Bus_i'
Bus_info.to_csv('./Pre_cal_data/Bus_info.csv')
Bus_info.head(5)

,vn_kv,max_vm_pu,min_vm_pu,type,zone,geo
Bus_i,,,,,,
1,12.66,1.0,1.0,b,1.0,"{""coordinates"": [-1.0272036165, 0.2320163609],..."
2,12.66,1.1,0.9,b,1.0,"{""coordinates"": [-1.7116237332, 1.1564808132],..."
3,12.66,1.1,0.9,b,1.0,"{""coordinates"": [-2.9808633284, 0.7559599298],..."
4,12.66,1.1,0.9,b,1.0,"{""coordinates"": [-3.522358388, 1.8171551243], ..."
5,12.66,1.1,0.9,b,1.0,"{""coordinates"": [-2.731321429, 2.6315460884], ..."


In [15]:
#Line
Line_column = ['from_bus','to_bus','r_ohm','x_ohm','c_nf','in_service','max_i_ka','max_loading_percent']
Line_info = pd.DataFrame(columns = Line_column)

Line_info['from_bus'] = net.line['from_bus'].values +1
Line_info['to_bus'] = net.line['to_bus'].values +1
Line_info['r_ohm'] = net.line['length_km'].values * net.line['r_ohm_per_km'].values
Line_info['x_ohm'] = net.line['length_km'].values * net.line['x_ohm_per_km'].values
Line_info['c_nf'] = net.line['length_km'].values * net.line['c_nf_per_km'].values
Line_info['in_service'] = net.line['in_service']
Line_info['max_i_ka'] = net.line['max_i_ka']
Line_info['max_loading_percent'] = net.line['max_loading_percent']

Line_info.index.name = 'Line_l'
Line_info.index = Line_info.index + 1
Line_info

Line_info.to_csv('./Pre_cal_data/Line_info.csv')
Line_info.head(5)

,from_bus,to_bus,r_ohm,x_ohm,c_nf,in_service,max_i_ka,max_loading_percent
Line_l,,,,,,,,
1,1,2,0.0922,0.0470,0.0,True,99999.0,100.0
2,2,3,0.4930,0.2511,0.0,True,99999.0,100.0
3,3,4,0.3660,0.1864,0.0,True,99999.0,100.0
4,4,5,0.3811,0.1941,0.0,True,99999.0,100.0
5,5,6,0.8190,0.7070,0.0,True,99999.0,100.0


In [16]:
#Gen Data
gen_columns = ['bus','in_service','vm_pu','p_mw','max_p_mw','min_p_mw','min_q_mvar','max_q_mvar']
gen_info = pd.DataFrame(columns = gen_columns)
try:
    # 기본 발전 데이터
    gen_info = net.gen[['bus','in_service','vm_pu','p_mw','max_p_mw','min_p_mw','min_q_mvar','max_q_mvar']]

    # 발전 비용함수 추가
    gen_info['cp0_eur']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cp0_eur']
    gen_info['cp1_eur_per_mw']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cp1_eur_per_mw']
    gen_info['cp2_eur_per_mw2']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cp2_eur_per_mw2']

    gen_info['cq0_eur']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cq0_eur']
    gen_info['cq1_eur_per_mvar']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cq1_eur_per_mvar']
    gen_info['cq2_eur_per_mvar2']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cq2_eur_per_mvar2']

    tmp = gen_info['bus'].values + 1
    gen_info['bus'] = tmp
    
except:
    print("Check genator info")

# Slack 모선 데이터 - Slack 모선이 발전기인 경우
slack_info = pd.DataFrame(net.ext_grid[['bus','in_service','vm_pu','max_p_mw','min_p_mw','min_q_mvar','max_q_mvar']])
slack_info['p_mw'] = 0
slack_info = slack_info[['bus','in_service','vm_pu','p_mw','max_p_mw','min_p_mw','min_q_mvar','max_q_mvar']]

# 발전 비용함수 추가
slack_info['cp0_eur']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cp0_eur']
slack_info['cp1_eur_per_mw']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cp1_eur_per_mw']
slack_info['cp2_eur_per_mw2']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cp2_eur_per_mw2']

slack_info['cq0_eur']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cq0_eur']
slack_info['cq1_eur_per_mvar']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cq1_eur_per_mvar']
slack_info['cq2_eur_per_mvar2']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cq2_eur_per_mvar2']

tmp = slack_info['bus'].values + 1
slack_info['bus']=tmp

try:
    gen_info = pd.concat([gen_info,slack_info])
    gen_info.sort_values(by=['bus'],axis=0,inplace=True)
    gen_info.reset_index(inplace=True,drop=True)
    gen_info.index = gen_info.index + 1
    gen_info.index.name = 'G_n'
except:
    gen_info = slack_info.copy()
    gen_info.reset_index(inplace=True,drop=True)
    gen_info.index = gen_info.index + 1
    gen_info.index.name = 'G_n'

gen_info.to_csv('./Pre_cal_data/Gen_info.csv')
gen_info.head(5)

Check genator info


,bus,in_service,vm_pu,p_mw,max_p_mw,min_p_mw,min_q_mvar,max_q_mvar,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
G_n,,,,,,,,,,,,,,
1,1,True,1.0,0,10.0,0.0,-10.0,10.0,0.0,20.0,0.0,0.0,0.0,0.0


In [17]:
#Load Data
Load_column = ['bus','p_mw','q_mvar','in_service']
Load_info = pd.DataFrame(columns = Load_column)
if 0 == net.bus['name'][0]:
    Load_info['bus']=net.load['bus'] + 1
else:
    Load_info['bus']=net.load['bus']
Load_info['p_mw'] = net.load['p_mw']
Load_info['q_mvar'] = net.load['q_mvar']
Load_info['in_service'] = net.load['in_service']

Load_info.index.name = 'Load_d'
Load_info.index=Load_info.index+1

Load_info.to_csv('./Pre_cal_data/Load_info.csv')
Load_info.head(5)


,bus,p_mw,q_mvar,in_service
Load_d,,,,
1,2,0.10,0.06,True
2,3,0.09,0.04,True
3,4,0.12,0.08,True
4,5,0.06,0.03,True
5,6,0.06,0.02,True


In [18]:
#Y Bus and Connectivity data
ymat = net._ppc['internal']['Ybus'].todense()
Y_mat_panda = pd.DataFrame(ymat)
if 0 == net.bus['name'][0]:
    bus_index = net.bus['name'].values + 1
else:
    bus_index = net.bus['name'].values 
    
Y_mat_panda.index = bus_index
Y_mat_panda.columns = bus_index
Y_mat_panda.to_csv('./Pre_cal_data/Ymat_panda.csv')

bus_multi_index = pd.MultiIndex.from_product(
    [bus_index, bus_index],
    names=["Bus_i", "Bus_j"]
)

Y_mat_info = pd.DataFrame(index=bus_multi_index,columns=['Bus_G','Bus_B'])

for i in bus_index:
    for j in bus_index:
        Y_mat_info.loc[(i,j),'Bus_G'] = np.real(Y_mat_panda.loc[i,j])
        Y_mat_info.loc[(i,j),'Bus_B'] = np.imag(Y_mat_panda.loc[i,j])

Y_mat_info.to_csv('./Pre_cal_data/Y_mat_info.csv')        
Y_mat_info.head(5)  

Bus_G      Bus_B
Bus_i Bus_j                       
1     1      137.979749 -70.336748
      2     -137.979749  70.336748
      3             0.0        0.0
      4             0.0        0.0
      5             0.0        0.0

Pandapower OPF 결과

In [19]:
pp.runopp(net, delta=1e-16,numba=False)

gen_mw_total = net.res_gen['p_mw'].sum() 
imports_mw_total = net.res_ext_grid['p_mw'].sum()

print('total gen MW:', gen_mw_total + imports_mw_total)
print('total imported gen MW:', imports_mw_total)
print('total local gen MW:', gen_mw_total)
print('total load MW:', net.res_load['p_mw'].sum())

total gen MW: 3.917677126455767
total imported gen MW: 3.917677126455767
total local gen MW: 0.0
total load MW: 3.715


In [20]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,3.917677,2.435141,-3.905437,-2.428901,0.012240,0.006240,0.210364,0.210364,0.210364,1.000000,0.000000,0.997032,0.014481,0.000210
1,3.444299,2.207822,-3.392508,-2.181444,0.051791,0.026379,0.187130,0.187130,0.187130,0.997032,0.014481,0.982938,0.096042,0.000187
2,2.362895,1.684201,-2.342995,-1.674065,0.019900,0.010135,0.134627,0.134627,0.134627,0.982938,0.096042,0.975456,0.161651,0.000135
3,2.222995,1.594065,-2.204296,-1.584542,0.018699,0.009524,0.127888,0.127888,0.127888,0.975456,0.161651,0.968059,0.228285,0.000128
4,2.144296,1.554542,-2.106047,-1.521524,0.038249,0.033018,0.124769,0.124769,0.124769,0.968059,0.228285,0.949658,0.133853,0.000125
5,1.095267,0.527888,-1.093353,-0.521559,0.001915,0.006329,0.058387,0.058387,0.058387,0.949658,0.133853,0.946173,-0.096474,0.000058
6,0.893353,0.421559,-0.888515,-0.419960,0.004838,0.001599,0.047612,0.047612,0.047612,0.946173,-0.096474,0.941328,-0.060403,0.000048
7,0.688515,0.319960,-0.684334,-0.316957,0.004181,0.003003,0.036782,0.036782,0.036782,0.941328,-0.060403,0.935059,-0.133484,0.000037
8,0.624334,0.296957,-0.620773,-0.294433,0.003561,0.002524,0.033719,0.033719,0.033719,0.935059,-0.133484,0.929244,-0.196014,0.000034
9,0.560773,0.274433,-0.560220,-0.274250,0.000554,0.000183,0.030640,0.030640,0.030640,0.929244,-0.196014,0.928384,-0.188761,0.000031


In [21]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent,geo
0,None,None,0,1,1.0,0.0922,0.0470,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
1,None,None,1,2,1.0,0.4930,0.2511,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
2,None,None,2,3,1.0,0.3660,0.1864,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
3,None,None,3,4,1.0,0.3811,0.1941,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
4,None,None,4,5,1.0,0.8190,0.7070,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
5,None,None,5,6,1.0,0.1872,0.6188,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
6,None,None,6,7,1.0,0.7114,0.2351,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
7,None,None,7,8,1.0,1.0300,0.7400,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
8,None,None,8,9,1.0,1.0440,0.7400,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
9,None,None,9,10,1.0,0.1966,0.0650,0.0,0.0,99999.0,1.0,1,ol,True,100.0,NaN
